In [1]:
import time
import requests 
import io
import hashlib
import os
from selenium import webdriver
import goto
from selenium.webdriver.common.keys import Keys
from PIL import Image, ImageDraw


In [2]:
def fetch_image_urls(query:str,wd,max_links_to_fetch:int):
    def scroll_to_bottom(wd):
        scroll_count=0
        while(scroll_count!=5):
            wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            scroll_count+=1
            time.sleep(2)
    
      # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))
    
    image_urls = set()
    img_count = 0
    results_start = 0
        
    while(len(image_urls)<max_links_to_fetch):
        # get all the thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector('img.rg_i.Q4LuWd')
        number_results = len(thumbnail_results)
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        if(number_results<max_links_to_fetch):
            scroll_to_bottom(wd)
            load_more_button = wd.find_element_by_xpath('//*[@id="islmp"]/div/div/div/div/div[1]/div[2]/div[2]/input')
            if load_more_button:
                load_more_button.click()
        # iterating through all the thumbnails and clicking them all to get the actual image
        for thumbnail_result in thumbnail_results[results_start:number_results]:
            if(len(image_urls)>=max_links_to_fetch):
                break
                
            else:
                try:
                    thumbnail_result.click()
                    time.sleep(1.5)
                except Exception as e:
                    print(e)
                    continue
                # getting the actual image from the thumbnail
                actual_img = wd.find_elements_by_css_selector('#Sva75c > div.DyeYj > div > div.dFMRD > div.pxAole > div.tvh9oe.BIB1wf > c-wiz > div.nIWXKc.JgfpDb > div.OUZ5W > div.zjoqD > div.qdnLaf.isv-id.b0vFpe > div > a > img')
                for actual_image in actual_img:
                    if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                        image_urls.add(actual_image.get_attribute('src'))
                        print(f'Extracting {len(image_urls)} of {max_links_to_fetch}')
                        if(len(image_urls)>=max_links_to_fetch):
                            break
                    else:
                        continue
    return image_urls




In [3]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")
        
        
def search_and_download(search_term:str,target_path:str,number_images):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path="C:\\Python\\scraping\\chromedriver.exe") as wd:
        res = fetch_image_urls(search_term, wd,number_images)
        
    for elem in res:
        persist_image(target_folder,elem)

In [4]:
# DRIVER_PATH = "C:\\Python\\scraping\\chromedriver.exe"
TARGET_PATH = "C:\\Python\\scraping\\New folder"
n = int(input('Enter no of queries'))
list = []
for i in range(0,n):
    ele = str(input())
    list.append(ele)
num = int(input('enter no of images required for each query'))
for ele in list:
    search_and_download(ele,target_path=TARGET_PATH,number_images=num)

Enter no of queries1
sachin
enter no of images required for each query10
Found: 48 search results. Extracting links from 0:48
Extracting 1 of 10
Extracting 2 of 10
Extracting 3 of 10
Extracting 4 of 10
Extracting 5 of 10
Extracting 6 of 10
Extracting 7 of 10
Extracting 8 of 10
Extracting 9 of 10
Extracting 10 of 10
SUCCESS - saved https://img.jagranjosh.com/imported/images/E/GK/sachin-records.png - as C:\Python\scraping\New folder\sachin\9164ba778a.jpg
SUCCESS - saved https://images.indianexpress.com/2020/04/tendulkar-1200.jpg - as C:\Python\scraping\New folder\sachin\70c51d3728.jpg
SUCCESS - saved https://images.news18.com/ibnlive/uploads/2021/04/sachin.jpg - as C:\Python\scraping\New folder\sachin\91aec3ac4a.jpg
SUCCESS - saved https://resources.pulse.icc-cricket.com/ICC/photo/2018/04/22/c19486c2-4e5b-48c4-82af-c6d0eebb7bd2/Main.jpg - as C:\Python\scraping\New folder\sachin\3cc0211d27.jpg
SUCCESS - saved https://akm-img-a-in.tosshub.com/businesstoday/images/story/202210/r-sixteen_nin